In [2]:
tile_info_path = "../tile_info" # file name tile_info_{model_name}_{dataset}.pt list
latency_path = "../latency" # file name latency_{model_name}_128.pt dict 
power_trace_path = "../power_trace" # file name power_trace_attck_{model_name}_{dataset}_normal.pt list

models = ["resnet18", "vgg16", "vgg8","densenet40", "densenet121", "alexnet"]
dataset = "cifar10"

##Power trace as both normal 

In [3]:
import torch 

ptrace = torch.load("/data/abommana/research_work/hw_security_CIM/v2_moo_framework/power_trace/combined_power_trace_vgg16_cifar10_normal.pt",weights_only=True)
print(ptrace)

[0.005819509986386402, 0.03851384343393147, 0.07579784665349872, 0.14970707967206837, 0.2884529056824744, 0.3874727210782468, 0.1940204313866794, 0.372963760542497, 0.1832384389270097, 0.36009196055196224, 0.36007409906554966, 0.35983190571963786, 0.36259049767935647, 0.36784002734888344, 0.36244723308440296, 0.3653815555440262, 0.3651949471382424, 0.36560455491375177, 0.3634069182731211, 0.3608926538432017, 0.358697159011662, 0.363400872592628, 0.35868337252717464, 0.3590420724645257, 0.3344298946608789, 0.33743553861817344, 0.335034694202058, 0.3354083046720363, 0.3380987821433693, 0.33488895956501363, 0.3275629201375879, 0.3304033552699722, 0.3306552887366153, 0.327883072400745, 0.3256801693637855, 0.32758721234817056, 0.31697963035507126, 0.3162403780438937, 0.3175715479588136, 0.316184331458807, 0.31683688910519703, 0.3160337392178364, 0.013736178992968053, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [4]:
len(ptrace)
ptrace[0:1]

[0.005819509986386402]

In [5]:
tile_info = torch.load("../latency/latency_vgg8_128.pt", weights_only=True)

print(tile_info)
len(tile_info)
# sum(tile_info)

{'layer1': 369497.0, 'layer2': 1210430.0, 'layer3': 370124.0, 'layer4': 495742.0, 'layer5': 129882.0, 'layer6': 132800.0, 'layer7': 3865.31, 'layer8': 823.711}


8

In [69]:
energy_model_dict = {}
tile_model_dict = {}
for model_name in models:
    tile_info = torch.load(f"{tile_info_path}/tile_info_{model_name}_{dataset}.pt",weights_only=True)
    latency_info = torch.load(f"{latency_path}/latency_{model_name}_128.pt",weights_only=True)
    power_trace = torch.load(f"{power_trace_path}/combined_power_trace_{model_name}_{dataset}_normal.pt",weights_only=True)

    ##We are trying to find the baseline energy
    total_tiles = sum(tile_info)
    tile_model_dict[model_name] = total_tiles
    print(power_trace)
    power_trace_model = power_trace[0:total_tiles]
    # print(power_trace_model)
    # print(len(power_trace_model))
    ##convert latency dict to list
    latency_list = []
    for key in latency_info.keys():
        latency_list.append(latency_info[key])
    tile_end_idx = 0
    energy_list = []
    print("tile_info", tile_info)
    print("sum of tiles", sum(tile_info))
    print("sum all latencies", sum(latency_list)*1e-6, "ms" )
    for layer in range(len(tile_info)):

        tile_start_idx = tile_end_idx
        tile_end_idx += tile_info[layer]
        # print("tile start and end idx", tile_start_idx, tile_end_idx)
        power_cummulative = sum(power_trace_model[tile_start_idx:tile_end_idx])
        # print(power_cummulative)
        energy_list.append(power_cummulative*latency_list[layer])
    print("Energy in mJ model_name", model_name," ", sum(energy_list)*1e-6, "\n")
    energy_model_dict[model_name] = sum(energy_list)

    ##SAVE THE BASELINE ENERGY 
torch.save(energy_model_dict,f"baseline_energy.pt" ) ##nJ
torch.save(tile_model_dict,f"tile_model_info.pt" ) ##nJ

[0.005848686358140549, 0.03345418354451977, 0.03721296402905137, 0.03831245319993468, 0.035826875433442186, 0.0767882737381151, 0.14411420868039132, 0.01235181011359673, 0.14553184403833003, 0.13341765971183778, 0.2927242951333523, 0.34974622944677247, 0.17870673096310347, 0.03354462517742068, 0.3516705358338542, 0.17312050356101244, 0.34008436076650395, 0.1681064516449347, 0.3498145196640864, 0.3434241830541752, 0.3481023576164618, 0.3198039293343201, 0.3217783856376074, 0.31878768137497826, 0.31643658035276456, 0.31818977565811946, 0.3145073644405231, 0.12204714279491455, 0.32022561618899925, 0.3285275124649517, 0.3185991682981141, 0.32536130465408786, 0.3226379768744111, 0.32414913093047215, 0.3559467016307637, 0.35258079683948307, 0.35083120633456855, 0.3496769080807455, 0.35116133870864286, 0.3506718660642393, 0.013643635098543019, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [7]:
print(energy_model_dict)

{'resnet18': 807891.4288485602, 'vgg16': 503653.9816272639, 'vgg8': 273205.20906389644, 'densenet40': 148234.58492611247, 'densenet121': 1305547.7037555377, 'alexnet': 300701.5872350505}


In [75]:
max_storage = 0.12
error_bits = 16
error_storage = max_storage*128*128*96*error_bits/1024/8 #KB
ppower_dram =  error_storage*20.7/64*1e-3
ppower_dram

0.11923199999999999

In [66]:
## Now you got the baselines, now get the energy for different tile EC components. 
##How will you get this done now??
import numpy as np
ec_energy_model_dict = {}
crossbar_info_path = "../crossbar_layers"
##for 64KB power is 20.7m W 
# Tile size  96 *128*128 * error bit width 7+7+3
##Required Error Storage --> 307.2KB (10 storage cells error bit is 16)--> 99.36 mW
max_storage = 0.12
error_bits = 4
error_storage = max_storage*128*128*96*error_bits/1024/8 #KB
power_ec = 0.21e-3
power_dram= error_storage*20.7/64*1e-3
tile_fraction_protection = [] ##givn a sinput
for model_name in models:
    crossbar_info = torch.load(f"{crossbar_info_path}/no_crossbar_layer_{model_name}_cifar10.pt", weights_only=True)
    # print(crossbar_info)
    tile_info = torch.load(f"{tile_info_path}/tile_info_{model_name}_{dataset}.pt",weights_only=True)
    total_tiles = sum(tile_info)
    latency_info = torch.load(f"{latency_path}/latency_{model_name}_128.pt",weights_only=True)
    latency_list = []
    for key in latency_info.keys():
        latency_list.append(latency_info[key])

    crosbar_list = []
    for i in range(total_tiles):
        tile_fraction_protection.append(np.random.rand(1).item())
    for key in crossbar_info.keys():
        crosbar_list.append(crossbar_info[key])
    
    ##Power of EC dram across each layer. 
    power_cummulative_dram = []
    tile_end_idx =0
    for layer in range(len(tile_info)):
        tile_start_idx = tile_end_idx
        tile_end_idx += tile_info[layer]
        pdram = 0
        tile_fraction_protection_layer = tile_fraction_protection[tile_start_idx:tile_end_idx]
        for i in range(tile_info[layer]):
            pdram += power_dram*max_storage*0.01
        power_cummulative_dram.append(pdram)
    power_total_ec = []
    for layer in range(len(tile_info)):
        pec = crosbar_list[layer]*power_ec + power_cummulative_dram[layer]
        power_total_ec.append(pec)

    energy_list = []

    for layer in range(len(tile_info)):
        energy_list.append(power_total_ec[layer]*latency_list[layer])
    
    print("energy_overhead in mJ for model",  model_name, " ", sum(energy_list)*1e-6)
    ec_energy_model_dict[model_name] = sum(energy_list)
    overhead_percentage = ec_energy_model_dict[model_name]/energy_model_dict[model_name] *100
    print("overhead percentgae for ec for model",model_name,"", round(overhead_percentage,3), "%")

    
torch.save(ec_energy_model_dict,f"ec_energy.pt") ##nJ





energy_overhead in mJ for model resnet18   0.04612924788732737
overhead percentgae for ec for model resnet18  5.71 %
energy_overhead in mJ for model vgg16   0.027799786783554215
overhead percentgae for ec for model vgg16  5.52 %
energy_overhead in mJ for model vgg8   0.014497890246048441
overhead percentgae for ec for model vgg8  5.307 %
energy_overhead in mJ for model densenet40   0.009967771781031937
overhead percentgae for ec for model densenet40  6.724 %
energy_overhead in mJ for model densenet121   0.07537313391903909
overhead percentgae for ec for model densenet121  5.773 %
energy_overhead in mJ for model alexnet   0.019345593855938877
overhead percentgae for ec for model alexnet  6.433 %


In [25]:
energy_overhead in mJ for model resnet18   0.09688714035325072
overhead percentgae for ec for model resnet18  11.993 %
energy_overhead in mJ for model vgg16   0.04961874978228496
overhead percentgae for ec for model vgg16  9.852 %
energy_overhead in mJ for model vgg8   0.033152669346210535
overhead percentgae for ec for model vgg8  12.135 %
energy_overhead in mJ for model densenet40   0.08028311997587277
overhead percentgae for ec for model densenet40  54.16 %
energy_overhead in mJ for model densenet121   0.2353811283052218
overhead percentgae for ec for model densenet121  18.029 %
energy_overhead in mJ for model alexnet   0.03145744318979533
overhead percentgae for ec for model alexnet  10.461 %

0.0630940246853974

In [12]:
import torch
import numpy as np

def compute_ec_overhead(
    tile_fraction_protection,
    model_name,
    dataset,
    tile_info_path="../tile_info",
    crossbar_info_path="../crossbar_layers",
    latency_path="../latency",
    baseline_energy_path="baseline_energy.pt",  # Path to the already-saved baseline energy dict
    max_storage=0.12,
    error_bits=4
):
    """
    Computes EC overhead for a given model and dataset. 
    Returns the EC overhead (in nJ) and overhead percentage relative to the baseline.

    Parameters:
    -----------
    tile_fraction_protection : list or array-like
        A list of tile-level fractions (0 to 1) indicating how much protection to apply at each tile.
        Must have length equal to sum of tile_info for the model.
    model_name : str
        Name of the model (e.g., "resnet18", "vgg16", etc.)
    dataset : str
        Name of the dataset (e.g., "cifar10")
    tile_info_path : str
        Directory containing the tile_info_{model_name}_{dataset}.pt files
    crossbar_info_path : str
        Directory containing the no_crossbar_layer_{model_name}_{dataset}.pt files
    latency_path : str
        Directory containing the latency_{model_name}_128.pt files
    baseline_energy_path : str
        File containing the baseline energy dictionary, keyed by model_name
    max_storage : float
        The fraction of some 'maximum' memory capacity used in your experiment (the original code uses 0.12).
        If you have multiple scenarios for different max_storages, adapt accordingly.
    error_bits : int
        Error bits used in the model (default is 4).

    Returns:
    --------
    ec_energy : float
        Total EC overhead energy in nJ
    overhead_percentage : float
        Overhead percentage relative to the baseline energy
    """

    # 1. Load the baseline energies (already computed & saved in baseline_energy.pt)
    baseline_energy_dict = torch.load(baseline_energy_path, weights_only=True)
    if model_name not in baseline_energy_dict:
        raise KeyError(f"Model '{model_name}' not found in baseline_energy dictionary.")
    baseline_energy = baseline_energy_dict[model_name]  # in nJ

    # 2. Compute error-storage-based power overhead
    #    (you can adapt these constants as needed, 
    #     but here we follow the original snippet)
    error_storage = max_storage * 128 * 128 * 96 * error_bits / 1024 / 8  # in KB
    power_ec   = 0.21e-3            # W  (i.e., J/ms) from code snippet
    power_dram = error_storage * 20.7 / 64 * 1e-3  # also W
    print(power_dram)

    # 3. Load crossbar info (no_crossbar_layer_{model_name}_{dataset}.pt)
    crossbar_info = torch.load(f"{crossbar_info_path}/no_crossbar_layer_{model_name}_{dataset}.pt", 
                               weights_only=True)
    crosbar_list = list(crossbar_info.values())  # same ordering as in code

    # 4. Load tile info and get total number of tiles
    tile_info = torch.load(f"{tile_info_path}/tile_info_{model_name}_{dataset}.pt", weights_only=True)
    total_tiles = sum(tile_info)
    if len(tile_fraction_protection) != total_tiles:
        raise ValueError(
            f"Length of tile_fraction_protection ({len(tile_fraction_protection)}) "
            f"must match total_tiles ({total_tiles})."
        )

    # 5. Load latency info and convert to list
    latency_info = torch.load(f"{latency_path}/latency_{model_name}_128.pt", weights_only=True)
    latency_list = list(latency_info.values())

    # 6. Compute DRAM overhead per layer, using tile_fraction_protection as needed
    tile_end_idx = 0
    power_cummulative_dram = []
    for layer in range(len(tile_info)):
        tile_start_idx = tile_end_idx
        tile_end_idx  += tile_info[layer]
        tile_fractions = tile_fraction_protection[tile_start_idx:tile_end_idx]
        
        # Original code example did not truly incorporate `tile_fractions` in the DRAM 
        # power calculation, but you can adapt as needed. Here we show the simplest
        # direct approach matching the snippet structure:
        pdram_layer = 0
        for frac in tile_fractions:
            # e.g., incorporate fraction here if you want fraction-based overhead:
            # pdram_layer += power_dram * max_storage * frac
            #
            # The snippet used a constant factor (0.01) for each tile:
            pdram_layer += power_dram * max_storage * frac
        
        power_cummulative_dram.append(pdram_layer)

    # 7. Compute total EC power per layer: crossbar + DRAM overhead
    power_total_ec = []
    for layer in range(len(tile_info)):
        # crossbar_list[layer] is the number of crossbars in that layer
        pec_layer = crosbar_list[layer] * power_ec + power_cummulative_dram[layer]
        power_total_ec.append(pec_layer)

    # 8. Compute total EC energy overhead across all layers
    ec_energy_list = []
    for layer in range(len(tile_info)):
        # Energy = Power * Time
        ec_energy_list.append(power_total_ec[layer] * latency_list[layer])
    ec_energy = sum(ec_energy_list)  # in nJ, if your power is in nJ/ms, be sure units are consistent

    # 9. Compute overhead percentage
    overhead_percentage = (ec_energy / baseline_energy) * 100.0

    return ec_energy, overhead_percentage


if __name__ == "__main__":
    # Example usage
    models = ["resnet18", "vgg16", "vgg8","densenet40", "densenet121", "alexnet"]
    dataset = "cifar10"
    tile_info_path="../tile_info"
    # Suppose we want overhead for resnet18 with random tile_fraction_protection:
    # (In a real scenario, you'd define or compute these fractions carefully.)
    for model_name in models:
        tile_info = torch.load("tile_model_info.pt", weights_only=True)
        # g = np.random.uniform(low=0.1, high=1, size=sum(tile_info))
        tile_info_layer = torch.load(f"{tile_info_path}/tile_info_{model_name}_{dataset}.pt", weights_only=True)
        tile_fraction_protection_example = np.random.uniform(low=0.1, high=1, size=sum(tile_info_layer)).tolist()  # e.g. 1000 tiles, just a placeholder
        print(tile_fraction_protection_example)
        ec_energy, overhead_pct = compute_ec_overhead(
            tile_fraction_protection=tile_fraction_protection_example,
            model_name=model_name,
            dataset=dataset,
            tile_info_path="../tile_info",
            crossbar_info_path="../crossbar_layers",
            latency_path="../latency",
            baseline_energy_path="baseline_energy.pt",  # adjust if needed
            max_storage=0.1,
            error_bits=25
        )
        print("model_name", model_name)
        print(f"EC Overhead (nJ): {ec_energy}")
        print(f"Overhead Percentage: {overhead_pct:.2f}%")


[0.8187353829358329, 0.30500958141839973, 0.685119580549779, 0.2987803109867717, 0.7358183216737569, 0.5825380914896231, 0.12123760343083305, 0.7476103948393562, 0.7714009372521431, 0.6013905363902395, 0.6304720726542248, 0.9029920205813863, 0.7612948945352938, 0.18053305259841962, 0.7767738501243845, 0.8519758321170082, 0.4063880654181127, 0.7087240374187935, 0.13374754263566888, 0.8399366637492568, 0.764376713013423, 0.3830142985423787, 0.8221525686112183, 0.10095012167194026, 0.858788648446056, 0.24381020216463062, 0.5429169069186098, 0.45548789297806935, 0.8785701855143282, 0.2780214289431723, 0.19452144216999007, 0.3075932087439484, 0.1026276197278053, 0.11058407644417828, 0.8564162931971527, 0.8942152747771543, 0.759266570866912, 0.17914236015675383, 0.7138747303686445, 0.7210606114065158, 0.7160990726473702]
0.09936000000000002
model_name resnet18
EC Overhead (nJ): 93381.45211472156
Overhead Percentage: 11.56%
[0.46883919865146373, 0.5466652263916243, 0.7799712928594169, 0.27032

In [1]:
a =[]

b= [3]
c =[4,5]

for i in [b,c]:
    a+=i

print(a)

[3, 4, 5]
